In [1]:
import pandas as pd
import time

In [4]:
SWurl = pd.read_csv('../data/url-IEEEData-dict2-subword.csv')

In [5]:
SWurl

,Unnamed: 0,word2,label,nurl
0,0,belajarprogramming com newpaypal loggy paypal...,1,"bela, jar, pro, gram, ming, com, new, pa, yp, ..."
1,1,ninedotlabs com att2 index.php,1,"nine, dot, lab, s, com, at, t, 2, index, ., php"
2,2,www almostfabulous com youker,0,"www, almost, fa, bu, lous, com, you, ker"
3,3,stbcomputer com autodesk components www.paypa...,1,"st, bc, omp, uter, com, auto, des, k, componen..."
4,4,members tripod com chang.ling,0,"members, trip, od, com, chang, ., ling"
...,...,...,...,...
95777,95777,www winehq com,0,"www, wine, h, q, com"
95778,95778,2933517 com js,1,"293, 35, 17, com, j, s"
95779,95779,imajinsmiles com Questions.html,1,"im, aj, ins, mi, les, com, questions, ., html"
95780,95780,thesecondhalfonline com wp-includes js tinymc...,1,"these, con, dha, lf, on, line, com, w, p, -, i..."


In [6]:
def add_word_to_dict(word,worddict):
    if (word not in worddict):
        worddict[word] = len(worddict)+1

In [7]:
worddict = dict()
for i in range(0,len(SWurl['nurl'])):
    a=SWurl['nurl'][i].split(', ')
    for j in range(0,len(a)):
        add_word_to_dict(a[j], worddict)

In [8]:
len(worddict)

14221

In [11]:
wordLength = 0
for i in range(0,len(SWurl['nurl'])):
    a=SWurl['nurl'][i].split(', ')
    if (len(a) > wordLength):
        wordLength = len(a)

In [10]:
wordLength

329

In [13]:
import numpy as np
X = np.zeros(shape=(len(SWurl['nurl']),wordLength))
for i in range(0, len(SWurl['nurl'])):
    a = SWurl['nurl'][i].split(', ')
    for j in range(0,len(a)):
        X[i, j] = worddict.get(a[j])

In [14]:
SWX = pd.DataFrame(X)
SWX = SWX.astype(int)
SWX

,0,1,2,3,4,5,6,7,8,9,...,319,320,321,322,323,324,325,326,327,328
0,1,2,3,4,5,6,7,8,9,10,...,0,0,0,0,0,0,0,0,0,0
1,18,19,20,21,6,22,23,24,25,15,...,0,0,0,0,0,0,0,0,0,0
2,27,28,29,30,31,6,32,33,0,0,...,0,0,0,0,0,0,0,0,0,0
3,34,35,36,37,6,38,39,40,41,27,...,0,0,0,0,0,0,0,0,0,0
4,68,69,70,6,71,15,72,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95777,27,6117,16,588,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95778,9549,452,240,6,79,21,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95779,84,3979,568,102,512,6,2850,15,101,0,...,0,0,0,0,0,0,0,0,0,0
95780,4655,668,8283,1696,322,913,6,172,199,45,...,0,0,0,0,0,0,0,0,0,0


In [17]:
X = SWX
X.shape

(95782, 329)

In [18]:
Y = SWurl['label']

In [19]:
Y.shape

(95782,)

In [20]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2, random_state=42)

C:\Users\User\anaconda3\lib\site-packages\scipy\__init__.py:173: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [21]:
Xtrain.shape, len(ytrain)

((76625, 329), 76625)

In [22]:
Xtest.shape, len(ytest)

((19157, 329), 19157)

In [23]:
vocab_size = 14222
maxLength = 329

In [24]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import SpatialDropout1D
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras.layers import GaussianNoise
from keras.layers import Bidirectional
from keras.regularizers import l1, l2, l1_l2

In [25]:
model = Sequential()

model.add(Embedding(vocab_size, 100, input_length=maxLength))
model.add(Dropout(0.4))
model.add(GaussianNoise(0.5))
model.add(Bidirectional(LSTM(128, activation='tanh', recurrent_dropout=0.4))) #, kernel_regularizer=l2(l2=0.01)
model.add(Dense(64, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

import time
start=time.time()

history = model.fit(Xtrain, ytrain, validation_data=(Xtest, ytest), epochs=10, verbose=1) 
print('time for training the model = {:.0f} sec'.format(time.time()-start))
# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))
print('Test Loss: %f' % (loss*100))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 329, 100)          1422200   
_________________________________________________________________
dropout (Dropout)            (None, 329, 100)          0         
_________________________________________________________________
gaussian_noise (GaussianNois (None, 329, 100)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               234496    
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,673,209
Trainable params: 1,673,209
Non-trainable params: 0
______________________________________________

In [26]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(Xtest)
print(confusion_matrix(ytest, np.rint(y_pred)))

[[9430  157]
 [ 256 9314]]


In [27]:
model.save("../saved_models/Subword-biLSTM_iter10.h5")

In [28]:
history2 = model.fit(Xtrain, ytrain, validation_data=(Xtest, ytest), epochs=5, verbose=1) 

print('time for training the model = {:.0f} sec'.format(time.time()-start))
# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))
print('Test Loss: %f' % (loss*100))

Epoch 1/5
2395/2395 [==============================] - 1831s 765ms/step - loss: 0.0356 - accuracy: 0.9874 - val_loss: 0.0706 - val_accuracy: 0.9780
Epoch 2/5
2395/2395 [==============================] - 1833s 765ms/step - loss: 0.0308 - accuracy: 0.9890 - val_loss: 0.0761 - val_accuracy: 0.9780
Epoch 3/5
2395/2395 [==============================] - 1836s 767ms/step - loss: 0.0281 - accuracy: 0.9901 - val_loss: 0.0755 - val_accuracy: 0.9789
Epoch 4/5
2395/2395 [==============================] - 1837s 767ms/step - loss: 0.0280 - accuracy: 0.9899 - val_loss: 0.0831 - val_accuracy: 0.9784
Epoch 5/5
2395/2395 [==============================] - 1838s 767ms/step - loss: 0.0249 - accuracy: 0.9909 - val_loss: 0.0778 - val_accuracy: 0.9796
time for training the model = 29415 sec
Test Accuracy: 97.958970
Test Loss: 7.779622


In [29]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(Xtest)
print(confusion_matrix(ytest, np.rint(y_pred)))

[[9403  184]
 [ 207 9363]]


In [30]:
model.save("../saved_models/Subword-biLSTM_iter15.h5")